# **Financial Machine Learning Assignment**

Student: B241413

Project choice: Forecasting Stock Volatility - Is stock volatility past-dependent?

Stock Choices:
1. Salesforce (CRM)
2. Goldman Sachs (GS)

In [1]:
#Import necessary libraries for analysis
import os
from os import listdir
import random
import tqdm
import numpy as np
import pandas as pd
import time

import tensorflow as tf
from tensorflow.python import keras
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras import backend as K

from keras.layers import Dense, Embedding, SimpleRNN
from keras.models import Sequential
from scipy import stats

from pprint import pprint
from pylab import plt, mpl
plt.style.use('seaborn')
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'
np.set_printoptions(suppress=True, precision=4)
os.environ['PYTHONHASHSEED'] = '0'

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import *
from sklearn.utils import resample
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from scipy import stats
import scipy.cluster.hierarchy as sch

%matplotlib inline

# These options determine the way floating point numbers are displayed.
np.set_printoptions(precision=4, suppress=True)

tf.random.set_seed(100)

2024-02-19 20:10:42.523831: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/var/folders/lg/tx14h9yd0y124jv8gq38m0mw0000gn/T/ipykernel_9118/3162438777.py:23: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


# Download stock from Yahoo Finance

In [2]:
### install and import yahoo finance
!pip install -U yfinance
import yfinance as yf

/Users/benblackwell/anaconda3/lib/python3.11/site-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [3]:
#### Define the stock Start and End Date ####
## you can define a longer time for your own use ##

start = pd.to_datetime('20000101',format='%Y%m%d')
end = pd.to_datetime('20231231',format='%Y%m%d')
days = (end-start).days
print('total days: ', days)

total days:  8765


In [4]:
#Reading the company listed stocks from the file 
# nasdaq_listed.csv contains all company tickers listed in nasdaq
df = pd.read_csv('/Users/benblackwell/Financial-Machine-Learning/nyse-listed.csv')
'''
find all company names listed on NASDAQ and download
the stoke prices from yahoo finance
'''
df.head()
Symbols = df['ACT Symbol'].tolist()

print('total number of tickers: ', len(Symbols))
print('first 10: ', Symbols[:10])



total number of tickers:  3298
first 10:  ['A', 'AA', 'AA$B', 'AAC', 'AAN', 'AAP', 'AAT', 'AAV', 'AB', 'ABB']


In [5]:
import pandas as pd
import yfinance as yf
import time

t0 = time.time()

# Define the stock tickers to retrieve prices for
tickers_to_download = ["GS", "CRM"]

# Define the time period for which you want to retrieve the prices
start = '2023-01-01'
end = '2024-01-01'

# Create empty dataframe
stock_final = pd.DataFrame()

# Empty list for the tickers that are not listed anymore
delisted_stock = []

# Iterate over each symbol
for ticker in tickers_to_download:
    print(f"Downloading data for {ticker}...", end=", ", flush=True)
    try:
        stock = yf.download(ticker, start=start, end=end, progress=False)
        if len(stock) == 0:
            delisted_stock.append(ticker)
        else:
            stock['Name'] = ticker
            stock_final = stock_final.append(stock, sort=False)
    except Exception:
        delisted_stock.append(ticker)

# Calculate total time taken
total_time = (time.time() - t0) / 60

print('\nTotal time:', total_time, 'minutes')
print('Total number of downloaded stocks:', len(stock_final['Name'].unique()))


Total time: 0.01214363177617391 minutes
Total number of downloaded stocks: 2


/var/folders/lg/tx14h9yd0y124jv8gq38m0mw0000gn/T/ipykernel_9118/372251870.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stock_final = stock_final.append(stock, sort=False)
/var/folders/lg/tx14h9yd0y124jv8gq38m0mw0000gn/T/ipykernel_9118/372251870.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stock_final = stock_final.append(stock, sort=False)


In [6]:
# Date index contains both date and time, we like to remove time information and use date information only

# make index as a normal column
stock_final.reset_index( inplace=True )

# remove time, leave dates only
stock_final['Date'] = pd.to_datetime(stock_final['Date'], format='%Y-%m-%d').dt.date

# set index as the date
stock_final.set_index('Date', inplace=True)

stock_final.head(10)

,Open,High,Low,Close,Adj Close,Volume,Name
Date,,,,,,,
2023-01-03,345.500000,348.600006,342.809998,346.220001,335.584076,1589700,GS
2023-01-04,348.029999,350.309998,345.079987,347.700012,337.018616,1881000,GS
2023-01-05,344.250000,345.140015,340.140015,343.760010,333.199677,1397800,GS
2023-01-06,347.299988,349.000000,340.950012,348.079987,337.386902,3097800,GS
2023-01-09,351.880005,356.500000,349.500000,353.000000,342.155792,1989000,GS
2023-01-10,352.640015,357.500000,350.500000,357.359985,346.381866,1610100,GS
2023-01-11,358.869995,365.500000,356.209991,364.480011,353.283112,2127000,GS
2023-01-12,366.239990,370.100006,362.570007,369.940002,358.575409,2103100,GS
2023-01-13,363.489990,374.350006,362.359985,374.000000,362.510681,3310200,GS


In [44]:
# !! if you have saved the data to your local place, no need to run this cell again !!
#*****************************************************************************************************************
print('total stock downloaded: ', stock_final['Name'].nunique())

# save to Google Drive, you should change the directory to your own
stock_final.to_csv('/Users/benblackwell/Financial-Machine-Learning/nyse_downloaded_stocks.csv')

total stock downloaded:  2


# Data Preparation 
As part of this project I have to calculate the volatility as the standard deviation of the 𝑁 daily arithmetic returns, but it's essential to note that this volatility should be computed based on returns within distinct, non-overlapping 𝑁-day intervals. 𝑁 can be five or ten days.

### Calculating Daily Returns

In [27]:
# Split the DataFrame into two based on the 'Name' column
gs_df = stock_final[stock_final['Name'] == 'GS']
crm_df = stock_final[stock_final['Name'] == 'CRM']

# Rename the 'Close' column for each
gs_df = gs_df.rename(columns={'Adj Close': 'GS_adj_close'}).drop(columns=['Name'])
crm_df = crm_df.rename(columns={'Adj Close': 'CRM_adj_close'}).drop(columns=['Name'])

# Now combine the two DataFrames to have 'GS_Close' and 'CRM_Close' columns aligned by 'Date'
combined_df = pd.merge(gs_df['GS_adj_close'], crm_df['CRM_adj_close'], left_index=True, right_index=True, how='inner')

combined_df.head(10)

,GS_adj_close,CRM_adj_close
Date,,
2023-01-03,335.584076,134.779999
2023-01-04,337.018616,139.589996
2023-01-05,333.199677,136.339996
2023-01-06,337.386902,140.509995
2023-01-09,342.155792,147.100006
2023-01-10,346.381866,147.440002
2023-01-11,353.283112,144.899994
2023-01-12,358.575409,149.600006
2023-01-13,362.510681,149.509995


In [29]:
# Calculate the simple return and the log return
# reference - Python for Finance

# For GS
combined_df['GS_simple_rtn'] = combined_df.GS_adj_close.pct_change()
combined_df['GS_log_rtn'] = np.log(combined_df.GS_adj_close/combined_df.GS_adj_close.shift(1))

# For CRM
combined_df['CRM_simple_rtn'] = combined_df.CRM_adj_close.pct_change()
combined_df['CRM_log_rtn'] = np.log(combined_df.CRM_adj_close/combined_df.CRM_adj_close.shift(1))
combined_df.head(10)

,GS_adj_close,CRM_adj_close,GS_simple_rtn,GS_log_rtn,CRM_simple_rtn,CRM_log_rtn
Date,,,,,,
2023-01-03,335.584076,134.779999,NaN,NaN,NaN,NaN
2023-01-04,337.018616,139.589996,0.004275,0.004266,0.035688,0.035066
2023-01-05,333.199677,136.339996,-0.011332,-0.011396,-0.023282,-0.023558
2023-01-06,337.386902,140.509995,0.012567,0.012488,0.030585,0.030127
2023-01-09,342.155792,147.100006,0.014135,0.014036,0.046901,0.045834
2023-01-10,346.381866,147.440002,0.012351,0.012276,0.002311,0.002309
2023-01-11,353.283112,144.899994,0.019924,0.019728,-0.017227,-0.017378
2023-01-12,358.575409,149.600006,0.014980,0.014869,0.032436,0.031921
2023-01-13,362.510681,149.509995,0.010975,0.010915,-0.000602,-0.000602


I have calculated the simple and the log returns in order to utilise either one later on in the analysis. 

There are two types of returns:
- Simple returns: They aggregate over assets; the simple return of a portfolio is the weighted sum of the returns of the individual assets in the portfolio.

- Log returns: They aggregate over time; it is easier to understand with the help of an example—the log return for a given month is the sum of the log returns of the days within that month. 


The difference between simple and log returns for daily/intraday data will be very small,
however, the general rule is that log returns are smaller in value than simple returns.